# FX Trading Backtesting

In [40]:
import oanda
import numpy as pd
import pandas as pd
import datetime as dt

In [41]:
data = oanda.get_candle_data('EUR_USD', 50, 'D')

df_data = {'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []}

for p in data['candles']:
    if p['complete'] == True:
        date = dt.datetime.strptime(p['time'][:10], "%Y-%m-%d")
        open_price = float(p['mid']['o'])
        high_price = float(p['mid']['h'])
        low_price = float(p['mid']['l'])
        close_price = float(p['mid']['c'])
        volume = int(p['volume'])
        
        df_data['Date'].append(date)
        df_data['Open'].append(open_price)
        df_data['High'].append(high_price)
        df_data['Low'].append(low_price)
        df_data['Close'].append(close_price)
        df_data['Volume'].append(volume)

df = pd.DataFrame(df_data)
df.set_index('Date', inplace=True)
df

,Open,High,Low,Close,Volume
Date,,,,,
2021-07-29,1.18846,1.19088,1.18516,1.18590,76567
2021-07-30,1.18592,1.18767,1.18538,1.18671,9383
2021-08-01,1.18623,1.18971,1.18590,1.18696,57883
2021-08-02,1.18698,1.18934,1.18536,1.18684,66493
2021-08-03,1.18686,1.18998,1.18328,1.18380,71416
2021-08-04,1.18381,1.18574,1.18280,1.18388,68530
2021-08-05,1.18390,1.18400,1.17544,1.17562,76019
2021-08-06,1.17560,1.17646,1.17560,1.17616,5729
2021-08-08,1.17643,1.17690,1.17406,1.17422,57924


## 1 Day Momentum

In [54]:
mom_period = 1

rets = []

for i in range(mom_period, len(df.index)-1):
    prev_date = df.index[i-mom_period]
    curr_date = df.index[i]
    next_date = df.index[i+1]
    
    prev_high = df.loc[prev_date,'High']
    curr_high = df.loc[curr_date, 'High']
    prev_low = df.loc[prev_date, 'Low']
    curr_low = df.loc[curr_date, 'Low']
    curr_close = df.loc[curr_date, 'Close']
    
    # trading condition
    # if current high is previous high, long 
    if curr_high > prev_high:
        print(curr_date, "long")
        next_close = df.loc[next_date,'Close']
        ret = ((next_close - curr_close) / curr_close) * 1000
        rets.append(ret)

    # if current low is previous low, short
    if curr_low < prev_low:
        print(curr_date, "short")
        next_close = df.loc[next_date, 'Close']
        
        

print(rets)
cumulative_ret = np.prod(rets)
cumulative_ret

2021-08-01 00:00:00
2021-08-03 00:00:00
2021-08-08 00:00:00
2021-08-10 00:00:00
2021-08-11 00:00:00
2021-08-12 00:00:00
2021-08-15 00:00:00
2021-08-18 00:00:00
2021-08-22 00:00:00
2021-08-23 00:00:00
2021-08-25 00:00:00
2021-08-26 00:00:00
2021-08-29 00:00:00
2021-08-30 00:00:00
2021-08-31 00:00:00
2021-09-01 00:00:00
2021-09-02 00:00:00
2021-09-09 00:00:00
2021-09-13 00:00:00
2021-09-19 00:00:00
2021-09-20 00:00:00
2021-09-22 00:00:00
[-0.10109860483935433, 0.06757898293637439, -2.2483009998126207, -0.8682033298151415, 5.281900120972536, -0.4067727665633091, -5.655858824328779, 1.2662126552820965, 0.6899841559193634, 1.08108108108117, 4.084202644521329, -0.5253970137112222, 0.8137386203740417, 3.4894554078088436, 1.4179369018079757, 2.064896755162108, -0.8747213928255434, -0.6684378860439023, -1.5395026222296844, 0.5289019313451725, 0.5797793428030744, -2.384155583181412]


1.797023662325002

## 2 Days Momentum

In [57]:
mom_period = 2

rets = []

for i in range(mom_period, len(df.index)-1):
    prev_date = df.index[i-mom_period]
    curr_date = df.index[i]
    next_date = df.index[i+1]
    
    prev_high = df.loc[prev_date,'High']
    curr_high = df.loc[curr_date, 'High']
    
    curr_close = df.loc[curr_date, 'Close']
    
    if curr_high > prev_high:
        print(curr_date)
        next_close = df.loc[next_date,'Close']
        ret = ((next_close - curr_close) / curr_close) * 1000
        rets.append(ret)

print(rets)
cumulative_ret = np.prod(rets)
cumulative_ret

2021-08-02 00:00:00
2021-08-03 00:00:00
2021-08-11 00:00:00
2021-08-12 00:00:00
2021-08-13 00:00:00
2021-08-22 00:00:00
2021-08-23 00:00:00
2021-08-24 00:00:00
2021-08-25 00:00:00
2021-08-26 00:00:00
2021-08-27 00:00:00
2021-08-29 00:00:00
2021-08-30 00:00:00
2021-08-31 00:00:00
2021-09-01 00:00:00
2021-09-02 00:00:00
2021-09-03 00:00:00
2021-09-13 00:00:00
2021-09-14 00:00:00
2021-09-20 00:00:00
2021-09-21 00:00:00
2021-09-22 00:00:00
2021-09-23 00:00:00
[-2.561423612281295, 0.06757898293637439, 5.281900120972536, -0.4067727665633091, -1.6955762415857045, 0.6899841559193634, 1.08108108108117, -0.6462475127974812, 4.084202644521329, -0.5253970137112222, 0.2543580004069448, 0.8137386203740417, 3.4894554078088436, 1.4179369018079757, 2.064896755162108, -0.8747213928255434, -0.6650335463124886, -1.5395026222296844, -3.744556837628637, 0.5797793428030744, 0.7498679209911822, -2.384155583181412, 0.37554838599545576]


1.8006486733192968

In [56]:
mom_period = 3

rets = []

for i in range(mom_period, len(df.index)-1):
    prev_date = df.index[i-mom_period]
    curr_date = df.index[i]
    next_date = df.index[i+1]
    
    prev_high = df.loc[prev_date,'High']
    curr_high = df.loc[curr_date, 'High']
    
    curr_close = df.loc[curr_date, 'Close']
    
    if curr_high > prev_high:
        print(curr_date)
        next_close = df.loc[next_date,'Close']
        ret = ((next_close - curr_close) / curr_close) * 1000
        rets.append(ret)

print(rets)
cumulative_ret = np.prod(rets)
cumulative_ret

2021-08-03 00:00:00
2021-08-12 00:00:00
2021-08-13 00:00:00
2021-08-15 00:00:00
2021-08-22 00:00:00
2021-08-23 00:00:00
2021-08-24 00:00:00
2021-08-25 00:00:00
2021-08-26 00:00:00
2021-08-27 00:00:00
2021-08-29 00:00:00
2021-08-30 00:00:00
2021-08-31 00:00:00
2021-09-01 00:00:00
2021-09-02 00:00:00
2021-09-03 00:00:00
2021-09-05 00:00:00
2021-09-14 00:00:00
2021-09-15 00:00:00
2021-09-21 00:00:00
2021-09-22 00:00:00
2021-09-23 00:00:00
[0.06757898293637439, -0.4067727665633091, -1.6955762415857045, -5.655858824328779, 0.6899841559193634, 1.08108108108117, -0.6462475127974812, 4.084202644521329, -0.5253970137112222, 0.2543580004069448, 0.8137386203740417, 3.4894554078088436, 1.4179369018079757, 2.064896755162108, -0.8747213928255434, -0.6650335463124886, -2.1733270436013434, -3.744556837628637, -2.891254804585061, 0.7498679209911822, -2.384155583181412, 0.37554838599545576]


-5.299369887201241